# Autoencoder

In [1]:
using Flux, Flux.Data.MNIST
using Flux: @epochs, onehotbatch, mse, throttle
using Base.Iterators: partition
using Juno: @progress
# using CuArrays

## Load data

In [2]:
# Encode MNIST images as compressed vectors that can later be decoded back into images.
imgs = MNIST.images();

In [3]:
# Partition into batches of size 1000
data = [float(hcat(vec.(imgs)...)) for imgs in partition(imgs, 1000)];

## Model

In [4]:
N = 32 # Size of the encoding

encoder = Dense(28^2, N, relu)
decoder = Dense(N, 28^2, relu)

m = Chain(encoder, decoder)

Chain(Dense(784, 32, NNlib.relu), Dense(32, 784, NNlib.relu))

## Loss function

In [5]:
loss(x) = mse(m(x), x)

loss (generic function with 1 method)

## Optimizer

Evaluation callback function

In [6]:
evalcb() = @show(loss(data[1]))
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

## Training

In [7]:
@epochs 10 Flux.train!(loss, params(m), zip(data), opt, cb=throttle(evalcb, 5))

┌ Info: Epoch 1
└ @ Main /home/pika/.julia/packages/Flux/T3PhK/src/optimise/train.jl:103


loss(data[1]) = 0.10143132390843942 (tracked)
loss(data[1]) = 0.06487017723375788 (tracked)
loss(data[1]) = 0.05109868361424779 (tracked)


┌ Info: Epoch 2
└ @ Main /home/pika/.julia/packages/Flux/T3PhK/src/optimise/train.jl:103


loss(data[1]) = 0.045234982304229045 (tracked)
loss(data[1]) = 0.03674250857078072 (tracked)
loss(data[1]) = 0.031240617984398342 (tracked)


┌ Info: Epoch 3
└ @ Main /home/pika/.julia/packages/Flux/T3PhK/src/optimise/train.jl:103


...

loss(data[1]) = 0.01578553885580495 (tracked)
loss(data[1]) = 0.015678063034902594 (tracked)
loss(data[1]) = 0.015551235833276241 (tracked)
loss(data[1]) = 0.015484794597874971 (tracked)
loss(data[1]) = 0.015381737458824592 (tracked)
loss(data[1]) = 0.01525126992440941 (tracked)


┌ Info: Epoch 9
└ @ Main /home/pika/.julia/packages/Flux/T3PhK/src/optimise/train.jl:103


loss(data[1]) = 0.015129474657067172 (tracked)
loss(data[1]) = 0.01500149688668255 (tracked)
loss(data[1]) = 0.014948678820193568 (tracked)
loss(data[1]) = 0.014930155856202363 (tracked)
loss(data[1]) = 0.014773730296627013 (tracked)


┌ Info: Epoch 10
└ @ Main /home/pika/.julia/packages/Flux/T3PhK/src/optimise/train.jl:103


loss(data[1]) = 0.014677877305216901 (tracked)
loss(data[1]) = 0.014593874336272829 (tracked)
loss(data[1]) = 0.014595970168856508 (tracked)
loss(data[1]) = 0.014503345691405847 (tracked)
loss(data[1]) = 0.014442656438763184 (tracked)


## Sample output

In [8]:
using Images

In [9]:
img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
  # 20 random digits
  before = [imgs[i] for i in rand(1:length(imgs), 20)]
  # Before and after images
  after = img.(map(x -> m(float(vec(x))).data, before))
  # Stack them all together
  hcat(vcat.(before, after)...)
end

sample (generic function with 1 method)

In [10]:
save("sample.jpg", sample())